***
Importando Librerias

In [2]:
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as ec

from IPython.display import clear_output
import csv

Se crea el Driver

In [3]:
options = uc.ChromeOptions()
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-infobars")
options.headless=False
driver = uc.Chrome(options=options)

***
Me posiciono exactamente donde Steam muestra las Ventas a Nivel Global

In [4]:
url = 'https://store.steampowered.com/search/?filter=globaltopsellers&ndl=1'
driver.get(url=url)

*** 
Hago Iteraciones de Scroll para poder que se cargue toda la Informacion (Todos los titulos)
Para preveer posibles interrupciones haré interrupciones cada 50 Scroll
Respetando un Tiempo para no sobrecargar el Servidor de Peticiones, dar Tiempo de que Cargue la Inf, y no me Bloquee la Ip por saturar el servidor

In [5]:
wait_ = WebDriverWait(driver,10)

In [ ]:
contador = 0
while True:
    # driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    window_height = driver.execute_script('return window.innerHeight;')
    scroll_position = int(window_height * 0.7)
    driver.execute_script(f'window.scrollBy(0, {scroll_position});')
    time.sleep(2)
    data_html = driver.find_elements(By.XPATH, '//*[@id="search_resultsRows"]/a')
    if len(data_html) == contador and contador > 9000:
        break
    else:
        contador = data_html


#(By.XPATH, '//*[@id="footer_logo"]/a/img')

In [ ]:
data_html = driver.find_elements(By.XPATH, '//*[@id="search_resultsRows"]/a')

In [30]:
len(data_html)

9092

***
Función que extrae Reseñas

In [11]:
def extraer_resenas(resena:str) -> tuple:
    index_1 = 0
    index_2 = 0
    for i in range(len(resena)):
        if resena[i] == '<':
            index_1 = i
            continue
        if resena[i].isdigit() and index_1 > 0:
            index_2 = i
            break
    index_3 = 0
    encontro_index3 = False
    for i in range(index_2+2,len(resena)):
        if resena[i].isdigit() and resena[i] != '0' and not encontro_index3:
            index_3 = i
            encontro_index3 = True
            continue
        if index_3 > 0 and resena[i] == ' ':
            index_4 = i + 1
            break

    critica_usuarios = resena[:index_1]
    porcentaje_aprobacion = resena[index_2:index_2+2]
    cantidad_reviews = resena[index_3:index_4-1]
    return (critica_usuarios, porcentaje_aprobacion, cantidad_reviews)

***
Funcion que Extrae Plataformas

In [12]:
def extrae_plataformas(elementos_plataformas: str) -> tuple[int, int, int]:
    windows = 0
    linux = 0
    mac = 0
    for i in range(3):
        try:
            plataforma = elementos_plataformas.find_element(By.XPATH, f'./span[{i+1}]').get_attribute('class')
            if 'win' in plataforma: windows = 1
            if 'linux' in plataforma: mac = 1
            if 'mac' in plataforma: linux = 1
        except:
            pass
        
    return (windows, mac, linux)

*** 
Ciclo que extrae toda la informacion

In [16]:
with open("data/ventas_actuales_steam.csv", "a", encoding="utf-8") as f:
    writer = csv.writer(f)
    data = []
    for i in range(len(data_html)):
        registro = {}
        try:
            titulo = data_html[i].find_element(By.XPATH, './div[2]/div[1]/span').text
        except:
            titulo = 'Sin Dato'
        try:
            elementos_plataformas = data_html[i].find_element(By.XPATH, './div[2]/div[1]/div')
            windows, mac, linux = extrae_plataformas(elementos_plataformas)
        except:
            windows = 0
            mac = 0
            linux = 0
        try:
            precio = data_html[i].find_element(By.XPATH, './div[2]/div[4]/div/div/div/div').text
        except:
            precio = 'Sin Dato'
        try:
            fecha = data_html[i].find_element(By.XPATH, './div[2]/div[2]').text
        except:
            fecha = 'Sin Dato'
        try:
            resena = data_html[i].find_element(By.XPATH, './div[2]/div[3]/span').get_attribute("data-tooltip-html")
            critica_usuarios,porcentaje_aprobacion,cantidad_reviews = extraer_resenas(resena) 
        except:
            critica_usuarios = 'Sin Dato'
            porcentaje_aprobacion = 'Sin Dato'
            cantidad_reviews = 'Sin Dato'

        # Voy escribiendo la informacion en un csv por si se corta el proceso no tener que volver a empezar de 0
        row = [titulo, critica_usuarios, porcentaje_aprobacion, cantidad_reviews, windows, mac, linux, fecha, precio]
        writer.writerow(row)

        registro['titulo'] = titulo
        registro['critica_usuarios'] = critica_usuarios
        registro['aprobacion'] = porcentaje_aprobacion
        registro['cantidad_reviews'] = cantidad_reviews
        registro['windows'] = windows
        registro['mac'] = mac
        registro['linux'] = linux
        registro['fecha_lanzamiento'] = fecha
        registro['precio_steam'] = precio
        data.append(registro)

        clear_output()
        print(f'Registro numero {i+1}')
        print(registro)

Registro numero 9137
{'titulo': 'Fatermyth', 'critica_usuarios': 'Muy positivas', 'aprobacion': '83', 'cantidad_reviews': '403', 'windows': 1, 'mac': 0, 'linux': 0, 'fecha_lanzamiento': '23 ABR 2024', 'precio_steam': '$5.79'}


In [17]:
len(data)

9137

***
Importo librerias para manipular Datos

In [18]:
import polars as pl

In [19]:
df = pl.DataFrame(data=data)

In [21]:
df.head()

titulo,critica_usuarios,aprobacion,cantidad_reviews,windows,mac,linux,fecha_lanzamiento,precio_steam
str,str,str,str,i64,i64,i64,str,str
"""Counter-Strike 2""","""Muy positivas""","""87""","""8,160,723""",1,1,0,"""21 AGO 2012""","""Gratuito"""
"""PUBG: BATTLEGROUNDS""","""Variadas""","""58""","""2,397,033""",1,0,0,"""21 DIC 2017""","""Gratuito"""
"""The First Descendant""","""Variadas""","""54""","""63,966""",1,0,0,"""30 JUN 2024""","""Gratuito"""
"""Dota 2""","""Muy positivas""","""81""","""2,278,393""",1,1,1,"""9 JUL 2013""","""Gratuito"""
"""Steam Deck""","""Sin Dato""","""Sin Dato""","""Sin Dato""",1,1,1,"""""","""Sin Dato"""


Creo el Ranking de Ventas, aprovechando que los titulos fueron introducidos en el orden de Ventas

In [23]:
df = df.with_row_index("ranking", offset=1)
df.head()

Trabajo en la columna `aprobacion`

In [26]:
df = df.with_columns(
    aprobacion = pl.when(pl.col('aprobacion') == 'Sin Dato')
                  .then(pl.lit('0'))
                  .otherwise(pl.col('aprobacion'))
)

In [42]:
df = df.with_columns(
    aprobacion = pl.col('aprobacion').str.strip_chars()
)

In [43]:
df = df.with_columns(
    aprobacion = pl.col('aprobacion').cast(pl.Int8)
)

Tranajo en la Columna `cantidad_reviews`

In [47]:
df = df.with_columns(
    cantidad_reviews = pl.col('cantidad_reviews').str.replace_all(',','')
)

In [49]:
df = df.with_columns(
    cantidad_reviews = pl.when(pl.col('cantidad_reviews') == 'Sin Dato')
                         .then(pl.lit(0))
                         .otherwise(pl.col('cantidad_reviews'))
)

In [52]:
df = df.with_columns(
    cantidad_reviews = pl.col('cantidad_reviews').cast(pl.Int64)
)

Creo Columna Categorica Ordinal para las Criticas de los Usuarios

In [58]:
df = df.with_columns(
    critica_categorica = pl.when(pl.col('critica_usuarios') == 'Extremadamente positivas')
                           .then(pl.lit(1))
                           .when(pl.col('critica_usuarios') == 'Muy positivas')
                           .then(pl.lit(2))
                           .when(pl.col('critica_usuarios') == 'Mayormente positivas')
                           .then(pl.lit(3))
                           .when(pl.col('critica_usuarios') == 'Positivas')
                           .then(pl.lit(4))
                           .when(pl.col('critica_usuarios') == 'Variadas')
                           .then(pl.lit(5))
                           .when(pl.col('critica_usuarios') == 'Mayormente negativas')
                           .then(pl.lit(6))
                           .when(pl.col('critica_usuarios') == 'Negativas')
                           .then(pl.lit(7))
                           .when(pl.col('critica_usuarios') == 'Muy negativas')
                           .then(pl.lit(8))
                           .when(pl.col('critica_usuarios') == 'Extremadamente negativas')
                           .then(pl.lit(9))
                           .otherwise(pl.lit(10))
)

Trabajando la Columna Fecha de Lanzamiento

In [63]:
df = df.with_columns(
    fecha_lanzamiento = pl.when(pl.col('fecha_lanzamiento') == 'Sin Dato')
                          .then(pl.lit(''))
                          .otherwise(pl.col('fecha_lanzamiento'))
)

Cambio Palabras de Español a Ingles para que puedan ser reconocidas por funciones de Fechas mas adelante

In [76]:
df = df.with_columns(
    fecha_ingles = (pl.col('fecha_lanzamiento').str.replace('ENE','JAN') 
                                               .str.replace('ABR','APR') 
                                               .str.replace('AGO','AUG') 
                                               .str.replace('DIC','DEC')
    )
)

In [82]:
df = df.with_columns(
    fecha = pl.col('fecha_ingles').str.strptime(pl.Date, format='%d %b %Y', strict=False)
)

#df = df.with_columns(pl.col('fecha_str').str.strptime(pl.Date, fmt='%d %b %Y').cast(pl.Datetime))

In [84]:
df.head(1)

ranking,titulo,critica_usuarios,aprobacion,cantidad_reviews,windows,mac,linux,fecha_lanzamiento,precio_steam,critica_categorica,fecha_ingles,fecha
u32,str,str,i8,i64,i64,i64,i64,str,str,i32,str,date
1,"""Counter-Strike 2""","""Muy positivas""",87,8160723,1,1,0,"""21 AGO 2012""","""Gratuito""",2,"""21 AUG 2012""",2012-08-21


In [86]:
df.write_csv('data/julio_2024_rank_steam.csv')